In [20]:
from llama_index.tools.tavily_research import TavilyToolSpec
import os
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from llama_index.llms.openai import OpenAI

In [21]:
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))

In [22]:
#llm = Ollama("llama3.2:1b")
llm = OpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [23]:
workflow = AgentWorkflow.from_tools_or_functions(
    tavily_tool.to_tool_list(),
    llm = llm,
    system_prompt = "You are a helpful assistant that can search the web for information"
)

In [24]:
from llama_index.core.agent.workflow import AgentStream

In [26]:
async def main():
    handler = workflow.run(user_msg="What's the weather like in Inhoaíba right now?")

    # handle streaming output
    async for event in handler.stream_events():
        if isinstance(event, AgentStream):
            print(event.delta, end="", flush=True)
        elif isinstance(event, AgentInput):
            print("Agent input: ", event.input)  # the current input messages
            print("Agent name:", event.current_agent_name)  # the current agent name
        elif isinstance(event, AgentOutput):
            print("Agent output: ", event.response)  # the current full response
            print("Tool calls made: ", event.tool_calls)  # the selected tool calls, if any
            print("Raw LLM response: ", event.raw)  # the raw llm api response
        elif isinstance(event, ToolCallResult):
            print("Tool called: ", event.tool_name)  # the tool name
            print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
            print("Tool output: ", event.tool_output)  # the tool output            

    # print final output
    print(str(await handler))

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

Agent input:  [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are a helpful assistant that can search the web for information')]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="What's the weather like in Inhoaíba right now?")])]
Agent name: Agent


Agent output:  assistant: None
Tool calls made:  [ToolSelection(tool_id='call_2bqNhzh4ZIRLE1bUaLcIpgkZ', tool_name='search', tool_kwargs={'query': 'weather in Inhoa\\u00edba', 'max_results': 3})]
Raw LLM response:  {'id': 'chatcmpl-BJ3lPuUEAFjkJi03C7IlCKA7H1mj4', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None}], 'created': 1743882399, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_c66b5540ac', 'usage': None}
Tool called:  search
Arguments to the tool:  {'query': 'weather in Inhoa\\u00edba', 'max_results': 3}
Tool output:  [Document(id_='b423323c-860c-4ff9-a5b6-48303f9458a5', embedding=None, metadata={'url': 'https://climatempo.info/27708/inhoaiba'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n',

In [9]:
import nest_asyncio
nest_asyncio.apply()